In [7]:
import os
from collections import defaultdict
import pandas as pd

In [3]:
nli_data_dir = '/home/emrys/Data/csk_data/csk_nli_22_01/'
filenames = os.listdir(nli_data_dir)

## Relation prediction

In [4]:
filenames

['fold3_w_neg_test_lemma.tsv',
 'fold1_w_neg_test_lemma.tsv',
 'fold5_w_neg_test_lemma.tsv',
 'fold1_w_neg_train_lemma.tsv',
 'fold3_w_neg_train_lemma.tsv',
 'fold4_w_neg_test_lemma.tsv',
 'fold4_w_neg_train_lemma.tsv',
 'fold2_w_neg_train_lemma.tsv',
 'fold5_w_neg_train_lemma.tsv',
 'fold2_w_neg_test_lemma.tsv']

In [24]:
with open('relations.txt', 'w') as f:
    for o in rels_set: f.write(o + '\n')

In [19]:
rels_set = set()
count = 0
for fn in filenames:
    with open(f'{nli_data_dir}/{fn}') as f:
        lines = f.readlines()
    dd = defaultdict(list)
    for l in lines:
        l = l.strip()
        dlg, a1, rel, a2, label = l.split('\t')
        label = rel if label == 'entailment' else 'NULL'
        dd['\t'.join([dlg, a1, a2])].append(label)
        rels_set.add(label)
    
    
    triplets = []
    for k, v in dd.items():
        if len(v) > 1:
            v = [o for o in v if o != 'NULL']
        dlg, a1, a2 = k.split('\t')            
        for vv in v:
            triplets.append([count, dlg, f'{a1} [SEP] {a2}', vv])
            count += 1
    df = pd.DataFrame(triplets, columns=['id', 'sentence1', 'sentence2', 'label'])
    df.to_csv(fn[:-4] + '.csv', index=False)

In [23]:
rels_set = set()
count = 0
for fn in filenames:
    with open(f'{nli_data_dir}/{fn}') as f:
        lines = f.readlines()
    dd = defaultdict(list)
    for l in lines:
        l = l.strip()
        dlg, a1, rel, a2, label = l.split('\t')
        if label == 'entailment': label = rel
        else: continue
        dd['\t'.join([dlg, a1, a2])].append(label)
        rels_set.add(label)
    
    
    triplets = []
    for k, v in dd.items():
        if len(v) > 1:
            v = [o for o in v if o != 'NULL']
        dlg, a1, a2 = k.split('\t')            
        for vv in v:
            triplets.append([count, dlg, f'{a1} [SEP] {a2}', vv])
            count += 1
    
    df = pd.DataFrame(triplets, columns=['id', 'sentence1', 'sentence2', 'label'])
    fold_n = fn[4]
    file_type = 'train' if 'train' in fn else 'test'
    df.to_csv(f"fold{fold_n}_{file_type}_lemma.csv", index=False)